In [ ]:
from flask import Flask, request, render_template
import sqlite3
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

app = Flask(__name__)

# 下载NLTK数据
nltk.download('punkt')
nltk.download('stopwords')

# 数据库连接
def get_db_connection():
    conn = sqlite3.connect('database.db')
    conn.row_factory = sqlite3.Row
    return conn

# 提取关键词
def extract_keywords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    keywords = [word for word in words if word.isalnum() and word.lower() not in stop_words]
    return keywords

# 主页路由
@app.route('/')
def index():
    return render_template('index.html')

# 搜索路由
@app.route('/search', methods=['POST'])
def search():
    text = request.form['text']
    keywords = extract_keywords(text)
    
    conn = get_db_connection()
    cursor = conn.cursor()
    
    results = []
    for keyword in keywords:
        cursor.execute('SELECT * FROM items WHERE name LIKE ? OR location LIKE ?', ('%' + keyword + '%', '%' + keyword + '%'))
        items = cursor.fetchall()
        for item in items:
            results.append(dict(item))
    
    conn.close()
    
    return render_template('results.html', results=results)

if __name__ == '__main__':
    app.run(debug=True)